### Looks into total training time and time per epoch
Should probably have done this in test analysis so I could include the results in that table
but I didnt...

In [8]:
import os
import pandas as pd
import numpy as np

In [9]:
source_dir = "/home/vincent/Documenten/BachelorsProject/ft_artist"
target_dir = "/home/vincent/Documenten/BachelorsProject/GitHub_Repo/results/ft_artist"

models = {
    "vit_t_16": {"vit": True},
    "swin_t": {"vit": True},
    "beit_b_16": {"vit": True},
    "deit_t_16": {"vit": True},
    "vgg19": {"vit": False},
    "resnet50": {"vit": False},
    "efficientnetv2_t": {"vit": False},
    "convnext_t": {"vit": False}
}

In [10]:
def getDuration(timefile: str):
    """" Extracts the training time in seconds from the time file """
    with open(timefile) as f:
        tmp = f.read()
    prefix = "Training time: "
    return float(tmp[len(prefix):])

In [11]:
# For each model and dataset we add a [training_time, time_per_epoch] pair
for model in models:
    models[model]["pairs"] = []

for file in os.scandir(source_dir):
    if file.is_file() and "validation" in file.name:
        timefile = file.path.replace("validation.csv", "trainingtime.txt")
        if os.path.exists(timefile):
            for model in models:
                if model not in file.name:
                    continue
                epochs = len(pd.read_csv(file.path))
                duration = getDuration(timefile)
                models[model]["pairs"] += [[duration, duration / epochs]]
        else:
            print("ERROR! {timefile} did not exist!")
            break

In [12]:
for model in models:
    models[model]["time_mean"] = np.mean([x[0] for x in models[model]["pairs"]])
    models[model]["time_std"] = np.std([x[0] for x in models[model]["pairs"]])
    models[model]["time_median"] = np.median([x[0] for x in models[model]["pairs"]])
    models[model]["time_q1"] = np.quantile([x[0] for x in models[model]["pairs"]], q=0.25)
    models[model]["time_q3"] = np.quantile([x[0] for x in models[model]["pairs"]], q=0.75)

    models[model]["epoch_time_mean"] = np.mean([x[1] for x in models[model]["pairs"]])
    models[model]["epoch_time_std"] = np.std([x[1] for x in models[model]["pairs"]])
    models[model]["epoch_time_median"] = np.median([x[1] for x in models[model]["pairs"]])
    models[model]["epoch_time_q1"] = np.quantile([x[1] for x in models[model]["pairs"]], q=0.25)
    models[model]["epoch_time_q3"] = np.quantile([x[1] for x in models[model]["pairs"]], q=0.75)

In [13]:
table = []
for model in models:
    table += [[
        model,
        models[model]["time_mean"],
        models[model]["time_std"],
        models[model]["time_median"],
        models[model]["time_q1"],
        models[model]["time_q3"],
        models[model]["epoch_time_mean"],
        models[model]["epoch_time_std"],
        models[model]["epoch_time_median"],
        models[model]["epoch_time_q1"],
        models[model]["epoch_time_q3"]
    ]]

df = pd.DataFrame(table, columns=[
        "model",
        "time_mean",
        "time_std",
        "time_median",
        "time_q1",
        "time_q3",
        "epoch_time_mean",
        "epoch_time_std",
        "epoch_time_median",
        "epoch_time_q1",
        "epoch_time_q3"
])

df

,model,time_mean,time_std,time_median,time_q1,time_q3,epoch_time_mean,epoch_time_std,epoch_time_median,epoch_time_q1,epoch_time_q3
0,vit_t_16,427.681926,53.236231,428.133527,384.462279,483.447611,11.999977,0.148474,12.014446,11.892598,12.086190
1,swin_t,927.674455,280.595023,831.624731,700.818536,1050.152439,29.136545,0.252906,29.179679,29.170901,29.200772
2,beit_b_16,2053.904807,402.769464,2028.606866,1800.983750,2145.800716,64.216008,0.547572,64.320848,63.892086,64.448876
3,deit_t_16,417.754194,102.806080,476.952063,310.928977,487.539099,12.123981,0.183986,12.112415,11.958807,12.188477
4,vgg19,913.748951,193.973332,852.182516,777.005222,1055.258077,33.845229,0.186118,33.782836,33.691020,34.040583
5,resnet50,942.363760,311.835507,875.903372,698.948537,1264.154510,22.444116,0.114097,22.459061,22.378441,22.546727
6,efficientnetv2_t,617.016915,89.853574,570.571835,558.733942,668.860374,26.346739,0.332980,26.462452,25.975367,26.606378
7,convnext_t,976.644995,122.330632,899.660628,867.383000,1125.559203,43.208254,0.188321,43.290739,43.238030,43.302367


In [14]:
df.to_csv(os.path.join(target_dir, "durations.csv"), index=False)